# d7046e ANN course project , Group 6

The project goal is to implement a classifier for clear/cloudy sky based satellite image data. 

In [4]:
# downloads and extract the data from, https://drive.google.com/drive/folders/1lRCIcQo9CqFRDhUd3aZRAA46k8nLL49J

import gdown
import zipfile
import os

FILE_ID = "19MBh9JIJTxYIPAeO7G5RML5_ddjJ1Cpa" # ID from the download url 

zip_path = "../data/skogsstyrelsen/data.zip" # path for the zip
extract_path = "../data/skogsstyrelsen/np_arrays/"

down_endpoint = "https://drive.google.com/uc?id=" # general drive download url

os.makedirs(extract_path, exist_ok=True)

gdown.download(f"{down_endpoint}{FILE_ID}", zip_path, quiet=False)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

os.remove(zip_path) # remove the zip, comment out if you want to keep the zip

print(f"Download and extraction completed. Files are in {extract_path}")

Downloading...
From: https://drive.google.com/uc?id=19MBh9JIJTxYIPAeO7G5RML5_ddjJ1Cpa
To: c:\Users\moell\dev\dev_uni\d7046e\project\d7046e_spaceDataAnalysis_project\data\skogsstyrelsen\data.zip
100%|██████████| 4.79M/4.79M [00:00<00:00, 10.4MB/s]


Download and extraction completed. Files are in ../data/skogsstyrelsen/np_arrays/
